In [ ]:
# Importando todas as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

%matplotlib inline

In [ ]:
# Importando o banco de dados
dados = pd.read_csv('Dados.csv', sep=';')
dados = dados.drop(['Temperatura_ensaio', 'Operacoes_medicoes', 'Operacoes_anual', 'Nivel de atrito'], axis=1)
dados.head() # Exibe as 5 primeiras linhas da matriz

In [ ]:
X, Y = dados.iloc[:, :6], dados.iloc[:, 6]

### Estatística descrtiva dos dados de entrada

In [ ]:
X.describe().round(2)

### Estatística descrtiva dos dados de saída

In [ ]:
Y.describe().round(2)

### Dividindo o conjunto de dados de entrada, X, e de saída, Y

### Dividindo aleatoriamente os conjuntos de dados entre 90% para Treinamento e 10% para Teste


Mais informações sobre a divisão de dados train_test_split:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
print('Matriz X_train:', X_train.shape)
print('Matriz X_test:', X_test.shape)
print('Matriz y_train:', y_train.shape)
print('Matriz y_test:', y_test.shape)

# Testando o pré-processamento de Standard Scaler
\begin{equation}
	z = \frac{(x - \overline{x})}{s}
\end{equation}
$z$: valor normalizado;<br>
$x$: valor a ser normalizado;<br>
$\overline{x}$: média;<br>
$s$: desvio padrão.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### ATENÇÃO: Na célula abaixo apenas as entradas são normalizadas

In [ ]:
# Pré-processando os dados de entrada utilizando o pré-processamento Standard Scaler
scaler_x = StandardScaler()
scaler_x.fit(X_train) # Ajustando os dados para a média e desvio padrão do conjunto de teste
X_train = scaler_x.transform(X_train) # Aplicando a transformação no conjunto de treinamento
X_test = scaler_x.transform(X_test) # O conjunto de teste é pré-processado de acordo com o conjunto de treinamento

### ATENÇÃO: Na célula abaixo as saídas também são normalizadas

In [ ]:
train = pd.concat([pd.DataFrame(X_train, columns=X.columns), pd.Series(y_train)], axis=1)
std_scaler = StandardScaler()
std_scaler.fit(train)
train = std_scaler.transform(train)
X_train, y_train = pd.DataFrame(train[:, :6], columns=X.columns), pd.Series(train[:, 6])

test = pd.concat([pd.DataFrame(X_test, columns=X.columns), pd.Series(y_test)], axis=1)
test = std_scaler.transform(test)
X_test, y_test = pd.DataFrame(test[:, :6], columns=X.columns), pd.Series(test[:, 6])

### Estatística descritiva dos dados de entrada após o pré-processamento dos dados de entrada do Treinamento

In [ ]:
pd.DataFrame(X_train, columns=X.columns).describe().round(2)

### Estatística descritiva dos dados de entrada após o pré-processamento dos dados de entrada do Teste

In [ ]:
pd.DataFrame(X_test, columns=X.columns).describe().round(2)

**Foi utilizado o *Multilayer Perceptron Regressor (MLP Regressor)* da biblioteca Scikit-Learn para o desenvolvimento dos modelos**

Mais informações sobre o MLP Regressor podem ser encontradas na página:<br> 
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

# Treinando 1 camada oculta

Neurônios: 1 a 100<br>
Função de ativação: Linear Retificada (ReLU)<br>
Solver: LBFGS<br>
Alpha: 0.003, 0.01, 0.03, 0.3, 0.5<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 11 (inicialização aleatória dos pesos)<br>
NOTA: o hiperparâmetro **Random state** possibilita reproduzir exatamente os mesmos resultados alcançados

***Para alterar a função de ativação deve-se alterar o hiperparâmetro 'activation' na linha 22 da célula abaixo, as seguintes funções de ativação estão disponíveis: identity (linear), logistic (sigmóide), tanh (tangente hiperbólica) e relu (linear retificada)***

In [ ]:
# Nesta célula são feitas 3 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) alpha;
# iii) random state.

inicio = time.time()

# Arquitetura da rede
alpha = [] # Alphas da rede

random_state = [] # Inicialização dos pesos

quantidade_arq_testadas = 0

armazenando_score_teste_max = 0

for i in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios em uma camada oculta
    for k in [0.003, 0.01, 0.03, 0.3, 0.5]:
        for z in np.arange(0, 11):
            reg_1 = MLPRegressor(
            hidden_layer_sizes=(i,),
            activation='relu',
            solver='lbfgs',
            alpha=k,
            random_state=z,
            max_iter=1000)
    
            reg_1.fit(X_train, y_train)
        
            armazenando_score_teste = reg_1.score(X_test, y_test)
        
            quantidade_arq_testadas += 1
                
            ultimo_neuronio_camada_1 = i # para saber onde parou, caso precisa interromper a execução
            
            ultimo_alpha = k # para saber onde parou, caso precisa interromper a execução
            
            ultimo_random_state = z # para saber onde parou, caso precisa interromper a execução
                
            print('Cam 1: %.i' %i, 'Random State: %.i' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train, y_train).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
            if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                armazenando_score_teste_max = armazenando_score_teste
                mse_test = mean_squared_error(y_test, reg_1.predict(X_test))
                mae_test = mean_absolute_error(y_test, reg_1.predict(X_test))
                random_state = z
                neuronios_camada_1 = i
                alpha = k
            
            fim = time.time() 
            tempo_total = fim - inicio # calcula o tempo de execução em segundos

    
    
print('Tempo de execução: %.2f' %tempo_total)

### Apresentado o melhor resultado

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_std_1_camada_relu_Operacoes_remocoes_alpha_0.003_0.01_0.03_0.3_0.5.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                alpha,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #alpha #tempo_total #quantidade_arq_testadas',)

# Testando 2 camadas

Neurônios na 1ª camada oculta: 1 a 100<br>
Neurônios na 2ª camada oculta: 1 a 100<br>
Função de ativação: Linear Retificada (ReLU)<br>
Solver: lbfgs<br>
Alpha: 0.01, 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 10 (inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 4 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) neurônios na 2ª camada oculta;
# iii) alpha;
# iv) random state.
armazenando_score_teste = 0

armazenando_score_teste_max = 0

# Arquitetura da rede
inicio = time.time()

alpha = []

random_state = []

quantidade_arq_testadas = 0

for i in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios em uma camada oculta
    for k in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios na segunda camada oculta
        for j in [0.01, 0.1]:
            for z in np.arange(0, 10):
                reg_2 = MLPRegressor(
                hidden_layer_sizes=(i, k),
                activation='relu',
                solver='lbfgs',
                alpha=j,
                random_state=z,
                max_iter=1000)
        
                reg_2.fit(X_train, y_train)
            
                armazenando_score_teste = reg_2.score(X_test, y_test)
        
                quantidade_arq_testadas += 1
                
                ultimo_neuronio_camada_1 = i
                
                ultimo_neuronio_camada_2 = k
                
                ultimo_alpha = j
                
                ultimo_random_state = z
                
                print('Cam 1: %.i' %i, 'Cam 2: %.i' %k, 'Alpha: %.2f' %j, 'Random State: %.i' %z,'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train, y_train).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
                if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                    armazenando_score_teste_max = armazenando_score_teste
                    mse_test = mean_squared_error(y_test, reg_2.predict(X_test))
                    mae_test = mean_absolute_error(y_test, reg_2.predict(X_test))
                    random_state = z
                    neuronios_camada_1 = i
                    neuronios_camada_2 = k
                    alpha = j
                    random_state = z
                
                fim = time.time()
                
                tempo_total = fim - inicio
        
        
print('Tempo de execução: %.2f s' %tempo_total)

### Apresentando o melhor resultado

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_std_2_camada_relu_Operacoes_remocoes_alpha.txt', 
           np.c_[armazenando_score_teste_max,
                 mse_test,
                 mae_test,
                 random_state,
                 neuronios_camada_1,
                 neuronios_camada_2,
                 alpha,
                 random_state,
                 tempo_total,
                 quantidade_arq_testadas],
           fmt='%.4f',
           comments='2_camadas_de_1_a_100_neuronios_e_alpha_0.01_e_0.1_e_random_state_0_10 ',
          header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #neuronios_camada_2 #alpha #random_state #tempo_total #quantidade_arq_testadas',)

# Testando 1 camada

Neurônios na 1ª camada oculta: 1 a 100<br>
Função de ativação: Tangente Hiperbólica (Tanh)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 10 (inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) random state.
armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios em uma camada oculta
    for k in np.arange(0, 11):
        reg_1 = MLPRegressor(
        hidden_layer_sizes=(i,k),
        activation='tanh',
        solver='lbfgs',
        alpha=0.1,
        random_state=k,
        max_iter=1000)
    
        reg_1.fit(X_train, y_train)
    
        armazenando_score_teste = reg_1.score(X_test, y_test)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i # para saber onde parou, caso precisa interromper a execução
            
        ultimo_random_state = z # para saber onde parou, caso precisa interromper a execução
                
        print('Cam 1: %.i' %i, 'Random State: %.i' %k, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train, y_train).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test, reg_1.predict(X_test))
            mae_test = mean_absolute_error(y_test, reg_1.predict(X_test))
            random_state = k
            neuronios_camada_1 = i
            
        fim = time.time() 
        tempo_total = fim - inicio # calcula o tempo de execução em segundos
                   
print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_std_1_camada_tanh_Operacoes_remocoes.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                alpha,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #alpha #tempo_total #quantidade_arq_testadas',)

# Testando 2 camadas

Neurônios na 1ª camada oculta: 1 a 100<br>
Função de ativação: Tangente Hiperbólica (Tanh)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 (inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) neurônios na 2ª camada oculta.
armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios na primeira camada oculta
    for k in np.arange(1, 101):  # Iteração entre 1 a 100 neurônios na segunda camada oculta
        reg_2 = MLPRegressor(
        hidden_layer_sizes=(i, k),
        activation='tanh',
        solver='lbfgs',
        alpha=0.1,
        random_state=0,
        max_iter=1000)
        
        reg_2.fit(X_train, y_train)
        
        armazenando_score_teste = reg_2.score(X_test, y_test)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i
                
        ultimo_neuronio_camada_2 = k
                
        print('Cam 1: %.i' %i, 'Cam 2: %.i' %k, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_2.score(X_train, y_train).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test, reg_2.predict(X_test))
            mae_test = mean_absolute_error(y_test, reg_2.predict(X_test))
            neuronios_camada_1 = i
            neuronios_camada_2 = k
                
            fim = time.time()
                
            tempo_total = fim - inicio
        
print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_std_2_camada_tanh_Operacoes_remocoes.txt',
           np.c_[armazenando_score_teste_max,
                 mse_test,
                 mae_test,
                 neuronios_camada_1,
                 neuronios_camada_2,
                 tempo_total,
                 quantidade_arq_testadas],
           fmt='%.4f',
           comments='2_camadas_de_1_a_100_neuronios_e_alpha_0.01_e_0.1_e_random_state_0_10 ',
          header='#armazenando_score_teste_max #mse_test #mae_test #neuronios_camada_1 #neuronios_camada_2 #tempo_total #quantidade_arq_testadas',)

# Testando o pré-processamento MinMaxScaler
\begin{equation}
 z = \frac{(x - x_\textit{min}) \times (d_2 - d_1)}{(x_\textit{max} - x_\textit{min})} + d_1
\end{equation}
$z$: valor normalizado;

$x$: valor a ser normalizado;

$x_{max}$ e $x_{min}$: variação do valor de $x$;

$d_1$ e $d_2$: limite ao qual o valor $x$ será reduzido.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

### Dividindo o cojunto de dados em 90% Treinamento e 10% Teste

In [ ]:
X_train_mms, X_test_mms, y_train_mms, y_test_mms = train_test_split(X, Y, test_size=0.1, random_state=0)

### Pré-processamento dos dados
**NOTA: Os dados de saída (coeficiente de atrito) foram normalizados**

In [ ]:
train = pd.concat([pd.DataFrame(X_train_mms, columns=X.columns), pd.Series(y_train_mms)], axis=1)
mms_scaler = MinMaxScaler()
mms_scaler.fit(train)
train = mms_scaler.transform(train)
X_train_mms, y_train_mms = pd.DataFrame(train[:, :6], columns=X.columns), pd.Series(train[:, 6])

test = pd.concat([pd.DataFrame(X_test_mms, columns=X.columns), pd.Series(y_test_mms)], axis=1)
test = mms_scaler.transform(test)
X_test_mms, y_test_mms = pd.DataFrame(test[:, :6], columns=X.columns), pd.Series(test[:, 6])

### Estatítisca descritiva do conjunto de entrada do treinamento

In [ ]:
X_train_mms.describe().round(2)

### Estatítisca descritiva do conjunto de entrada do teste

In [ ]:
X_test_mms.describe().round(2)

## Testando 1 camada

Neurônios na 1ª camada oculta: 1 a 100<br>
Função de ativação: Tangente Hiperbólica (Tanh)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 9 (inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) random state.
armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for z in np.arange(0, 10):
        reg_1 = MLPRegressor(
        hidden_layer_sizes=(i,),
        activation='relu',
        solver='lbfgs',
        alpha=0.1,
        max_iter=1000,
        random_state=z,)
    
        reg_1.fit(X_train_mms, y_train_mms)
            
        armazenando_score_teste = reg_1.score(X_test_mms, y_test_mms)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i # para saber onde parou, caso precisa interromper a execução
            
        ultimo_random_state = z # para saber onde parou, caso precisa interromper a execução
                
        print('Cam 1: %.i' %i, 'Random State: %.i' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train_mms, y_train_mms).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test_mms, reg_1.predict(X_test_mms))
            mae_test = mean_absolute_error(y_test_mms, reg_1.predict(X_test_mms))
            random_state = z
            neuronios_camada_1 = i
        
        fim = time.time()
        
        tempo_total = fim - inicio
print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_mms_1_camada_tanh.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #tempo_total #quantidade_arq_testadas',)

## Testando 2 camadas

Neurônios na 1ª camada oculta: 1 a 100<br>
Neurônios na 2ª camada oculta: 1 a 100<br>
Função de ativação: Tangente Hiperbólica (Tanh)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0(inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) neurônios na 2ª camada oculta.

armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for k in np.arange(1, 101):
        reg_2 = MLPRegressor(
        hidden_layer_sizes=(i,k),
        activation='tanh',
        solver='lbfgs',
        alpha=0.1,
        max_iter=1000,
        random_state=0,)
        
        reg_2.fit(X_train_mms, y_train_mms)
        
        armazenando_score_teste = reg_2.score(X_test_mms, y_test_mms)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i
                
        ultimo_neuronio_camada_2 = k
                
        print('Cam 1: %.i' %i, 'Cam 2: %.i' %k, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_2.score(X_train_mms, y_train_mms).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test_mms, reg_2.predict(X_test_mms))
            mae_test = mean_absolute_error(y_test_mms, reg_2.predict(X_test_mms))
            neuronios_camada_1 = i
            neuronios_camada_2 = k
                
        fim = time.time()
                
        tempo_total = fim - inicio

print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
#np.savetxt('resultados_mms_2_camadas_tanh.txt',
np.savetxt('teste.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                neuronios_camada_2, 
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #neuronios_camada_2 #tempo_total #quantidade_arq_testadas',)

# Testando 1 camada

Neurônios na 1ª camada oculta: 1 a 100<br>
Função de ativação: Linear Retificada (ReLU)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 9(inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) random state.

armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for z in np.arange(0, 10):
        reg_1 = MLPRegressor(
        hidden_layer_sizes=(i,),
        activation='relu',
        solver='lbfgs',
        alpha=0.1,
        max_iter=1000,
        random_state=z,)
    
        reg_1.fit(X_train_mms, y_train_mms)
            
        armazenando_score_teste = reg_1.score(X_test_mms, y_test_mms)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i # para saber onde parou, caso precisa interromper a execução
            
        ultimo_random_state = z # para saber onde parou, caso precisa interromper a execução
                
        print('Cam 1: %.i' %i, 'Random State: %.i' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train_mms, y_train_mms).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test_mms, reg_1.predict(X_test_mms))
            mae_test = mean_absolute_error(y_test_mms, reg_1.predict(X_test_mms))
            random_state = z
            neuronios_camada_1 = i
        
        fim = time.time()
        
        tempo_total = fim - inicio
print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_mms_1_camada_relu.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #tempo_total #quantidade_arq_testadas',)

# Testando 2 camadas

Neurônios na 1ª camada oculta: 1 a 100<br>
Neurônios na 2ª camada oculta: 1 a 100<br>
Função de ativação: Linear Retificada (ReLU)<br>
Solver: LBFGS<br>
Alpha: 0.01 e 0.3<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 9(inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 3 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) neurônios na 2ª camada oculta;
# ii) alpha.

armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for k in np.arange(1, 101):
        for z in [0.01, 0.3]:
            reg_2 = MLPRegressor(
            hidden_layer_sizes=(i,k),
            activation='relu',
            solver='lbfgs',
            alpha=z,
            max_iter=1000,
            random_state=0,)
    
            reg_2.fit(X_train_mms, y_train_mms)
        
            armazenando_score_teste = reg_2.score(X_test_mms, y_test_mms)
        
            quantidade_arq_testadas += 1
                
            ultimo_neuronio_camada_1 = i
                
            ultimo_neuronio_camada_2 = k
                
            print('Cam 1: %.i' %i, 'Cam 2: %.i' %k, 'Alpha: %.3f' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_2.score(X_train_mms, y_train_mms).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
            if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                armazenando_score_teste_max = armazenando_score_teste
                mse_test = mean_squared_error(y_test_mms, reg_2.predict(X_test_mms))
                mae_test = mean_absolute_error(y_test_mms, reg_2.predict(X_test_mms))
                neuronios_camada_1 = i
                neuronios_camada_2 = k
                alpha = z
                
        fim = time.time()
                
        tempo_total = fim - inicio

print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_mms_2_camada_relu_alphas.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                neuronios_camada_1,
                neuronios_camada_2,
                alpha,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #neuronios_camada_1 #neuronios_camada_2 #alpha #tempo_total #quantidade_arq_testadas',)

# Testando o pré-processamento MaxAbsScaler


Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler

### Dividindo o cojunto de dados em 90% Treinamento e 10% Teste

In [ ]:
X_train_mas, X_test_mas, y_train_mas, y_test_mas = train_test_split(X, Y, test_size=0.1, random_state=0)

### Pré-processamento dos dados
**NOTA: Os dados de saída (coeficiente de atrito) foram normalizados**

In [ ]:
train = pd.concat([pd.DataFrame(X_train_mas, columns=X.columns), pd.Series(y_train_mas)], axis=1)
mas_scaler = MaxAbsScaler()
mas_scaler.fit(train)
train = mas_scaler.transform(train)
X_train_mas, y_train_mas = pd.DataFrame(train[:, :6], columns=X.columns), pd.Series(train[:, 6])

test = pd.concat([pd.DataFrame(X_test_mas, columns=X.columns), pd.Series(y_test_mas)], axis=1)
test = mas_scaler.transform(test)
X_test_mas, y_test_mas = pd.DataFrame(test[:, :6], columns=X.columns), pd.Series(test[:, 6])

### Estatítisca descritiva do conjunto de entrada do treinamento

In [ ]:
X_train_mas.describe().round(2)

### Estatítisca descritiva do conjunto de entrada do teste

In [ ]:
X_test_mas.describe().round(2)

## Testando 1 camada

Neurônios na 1ª camada oculta: 1 a 100<br>
Função de ativação: Tangente Hiperbólica (Tanh)<br>
Solver: LBFGS<br>
Alpha: 0.1<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 9 (inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 2 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) random state.
armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for z in np.arange(0, 10):
        reg_1 = MLPRegressor(
        hidden_layer_sizes=(i,),
        activation='relu',
        solver='lbfgs',
        alpha=0.1,
        max_iter=1000,
        random_state=z,)
    
        reg_1.fit(X_train_mas, y_train_mas)
            
        armazenando_score_teste = reg_1.score(X_test_mas, y_test_mas)
        
        quantidade_arq_testadas += 1
                
        ultimo_neuronio_camada_1 = i # para saber onde parou, caso precisa interromper a execução
            
        ultimo_random_state = z # para saber onde parou, caso precisa interromper a execução
                
        print('Cam 1: %.i' %i, 'Random State: %.i' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_1.score(X_train_mas, y_train_mas).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
        if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
            armazenando_score_teste_max = armazenando_score_teste
            mse_test = mean_squared_error(y_test_mas, reg_1.predict(X_test_mas))
            mae_test = mean_absolute_error(y_test_mas, reg_1.predict(X_test_mas))
            random_state = z
            neuronios_camada_1 = i
        
        fim = time.time()
        
        tempo_total = fim - inicio
print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_mas_1_camada_tanh.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                random_state,
                neuronios_camada_1,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #random_state #neuronios_camada_1 #tempo_total #quantidade_arq_testadas',)

# Testando 2 camadas

Neurônios na 1ª camada oculta: 1 a 100<br>
Neurônios na 2ª camada oculta: 1 a 100<br>
Função de ativação: Linear Retificada (ReLU)<br>
Solver: LBFGS<br>
Alpha: 0.01 e 0.3<br>
Nº máx. de iterações: 1.000<br>
Random state: 0 a 9(inicialização de pesos)<br>

In [ ]:
# Nesta célula são feitas 3 iterações simultâneas:
# i) neurônios na 1ª camada oculta;
# ii) neurônios na 2ª camada oculta;
# ii) alpha.

armazenando_score_teste = 0

armazenando_score_teste_max = 0

inicio = time.time()

quantidade_arq_testadas = 0

for i in np.arange(1, 101):
    for k in np.arange(1, 101):
        for z in [0.01, 0.3]:
            reg_2 = MLPRegressor(
            hidden_layer_sizes=(i,k),
            activation='relu',
            solver='lbfgs',
            alpha=z,
            max_iter=1000,
            random_state=0,)
    
            reg_2.fit(X_train_mas, y_train_mas)
        
            armazenando_score_teste = reg_2.score(X_test_mas, y_test_mas)
        
            quantidade_arq_testadas += 1
                
            ultimo_neuronio_camada_1 = i
                
            ultimo_neuronio_camada_2 = k
                
            print('Cam 1: %.i' %i, 'Cam 2: %.i' %k, 'Alpha: %.3f' %z, 'Quantidade_arq_testadas: %.i' %quantidade_arq_testadas, 'R² treino: %.2f' %(reg_2.score(X_train_mas, y_train_mas).round(2)*100), 'R² teste: %.2f' %(armazenando_score_teste*100))
                
            if armazenando_score_teste > armazenando_score_teste_max: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                armazenando_score_teste_max = armazenando_score_teste
                mse_test = mean_squared_error(y_test_mas, reg_2.predict(X_test_mas))
                mae_test = mean_absolute_error(y_test_mas, reg_2.predict(X_test_mas))
                neuronios_camada_1 = i
                neuronios_camada_2 = k
                alpha = z
                
        fim = time.time()
                
        tempo_total = fim - inicio

print('Tempo de execução: %.2f' %tempo_total)

### Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_test)
print('Erro Médio Absoluto do teste: %.3f' %mae_test)

### Salvando os resultados em arquivo TXT

In [ ]:
np.savetxt('resultados_mas_2_camada_relu_alphas.txt',
           np.c_[armazenando_score_teste_max,
                mse_test,
                mae_test,
                neuronios_camada_1,
                neuronios_camada_2,
                alpha,
                tempo_total,
                quantidade_arq_testadas],
           fmt='%.4f',
           comments='',           
           header='#armazenando_score_teste_max #mse_test #mae_test #neuronios_camada_1 #neuronios_camada_2 #alpha #tempo_total #quantidade_arq_testadas',)